# Best Model Algorithmic Trading - By Foivos Gaitantzis

#### This section of the program is dedicated to building an algorithmic trading bot by making use of the Machine Learning Algorithm with the Lowest RMSE (Linear Regression).

# Best Model Algorithmic Trading

### Load the Libraries

In [95]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

### Define the Feature Engineering/Machine Learning Parameters & Utility Functions

In [96]:
#Set the filename
filename = 'aapl'

#Set the Final Hyperparameters Acquired in Linear Regression 
Columns = ['Adj Close', 'Volume', 'compoundFT', 'compoundNYTimes']
timestep = 4
polynomial_degree = 1

#Define the initial Money Amount for Trading
Money = 10000

In [97]:
#Define a DataFrame that Holds Curent Stocks
Stocks = pd.DataFrame({'Date': [], 'Share Count': [], 'Buy Price': []})

#Buy/Sell Stock Utility Function
def BuySellStock():
    #Define Global Variables
    global Money
    global Stocks
    global CurrentDate
    #Get the Max Change in Real Price
    DateToday = pd.to_datetime(CurrentDate, format='%Y-%m-%d').date()
    MaxChange = Prediction['Predicted Change'].max()
    #How many stocks can I buy using my budget?
    StockCount = int(Money / CurrentPrice)
    #If the Predicted Price for Tomorrow is Higher than Today: Buy
    if (NextDayPrediction > CurrentPrice):
        #Calculate the difference between Today's Price and Tomorrow's Predicted
        Change = NextDayPrediction - CurrentPrice
        #The Higher the difference the more stocks to buy up to StockCount
        StocksBuy = int((Change * MaxChange) / (((MaxChange * MaxChange) - 1) / StockCount) + 1 )
        if (StocksBuy > 0):
            #Buy Stocks, Append to DataFrame and Deduct Money
            Stocks = Stocks.append(pd.DataFrame({'Date': [CurrentDate], 'Share Count': [StocksBuy], 'Buy Price': [CurrentPrice]}), ignore_index=True)
            Money = Money - (CurrentPrice * StocksBuy)
            print(str(DateToday)+': '+str(StocksBuy)+' Shares Bought for: $'+str(round(CurrentPrice, 2))+' Each, Total: $'+str(round(CurrentPrice*StocksBuy, 2)))
    #If the Predicted Price for Tomorrow is Lower than Today: Sell
    elif (NextDayPrediction < CurrentPrice):
        #Loop through all Stocks Bought
        for v in range(len(Stocks)-1, -1, -1):
            #If Stock Price of Tomorrrow Lower than Buy Price of Stock: Sell
            if (Stocks['Buy Price'].values[v] > CurrentPrice):
                #Sell Stocks, Delete from DataFrame and Append Money
                Money = Money + (CurrentPrice * Stocks['Share Count'].values[v])
                Profit = (CurrentPrice*Stocks['Share Count'].values[v]) - (Stocks['Buy Price'].values[v]*Stocks['Share Count'].values[v])
                print(str(DateToday)+': '+str(int(Stocks['Share Count'].values[v]))+' Shares Sold for: $'+str(round(CurrentPrice, 2))+' Each. Profit Made: $'+str(round(Profit, 2)))
                Stocks.drop(Stocks.index[v], inplace=True)

### Load the Data File & Perform Model Feature Engineering

In [98]:
#Load the Daily Stock Data
Data = pd.read_csv('files/'+filename+'_Stock_Data_Full.csv')

#Set the Date Column Type to Datetime
Data['Date'] = pd.to_datetime(Data['Date'], format='%Y-%m-%d')

#Create copies of the Data for Modification
Data_Edit = Data.copy()
Columns_Edit = Columns.copy()

#Create new to columns of Historical Data according to the timestep
for step in range(timestep):
    Columns_Edit.append('Adj Close'+str(step))
    Data_Edit['Adj Close'+str(step)] = Data_Edit['Adj Close']
    Data_Edit['Adj Close'+str(step)] = Data_Edit['Adj Close'+str(step)].shift(step)

#Remove all NaN values in Adj Close, Adj Close(0 - timestep), Predicted Adj Close, RSI and MACD
Data_Edit.dropna(inplace = True)

#Display the first 5 rows of the Modified Data Set
Data_Edit.head()

,Date,compoundFT,compoundNYTimes,compoundBS,Close,Volume,Adj Close,RSI,MACD,Predicted Adj Close,Adj Close0,Adj Close1,Adj Close2,Adj Close3
7,2016-01-08,0.203480,0.043943,0.137486,96.959999,7.079800e+07,90.391907,14.668720,-0.370537,90.879794,90.391907,89.916473,93.878586,95.752419
8,2016-01-09,0.377325,-0.008900,0.104625,97.483332,6.377847e+07,90.879794,14.832344,-0.384101,91.367681,90.879794,90.391907,89.916473,93.878586
9,2016-01-10,0.515333,0.271481,0.135400,98.006666,5.675893e+07,91.367681,14.886746,-0.364081,91.855568,91.367681,90.879794,90.391907,89.916473
10,2016-01-11,0.233810,0.375018,0.179233,98.529999,4.973940e+07,91.855568,14.913921,-0.321599,93.188705,91.855568,91.367681,90.879794,90.391907
11,2016-01-12,0.155050,0.201992,0.397150,99.959999,4.915420e+07,93.188705,19.135019,-0.221146,90.792793,93.188705,91.855568,91.367681,90.879794


In [99]:
#Split the Data into Training and Testing (Train: 80% & Testing: 20%)
train, test = train_test_split(Data_Edit, train_size = 0.8, test_size = 0.2, shuffle = False)

#Initiate two Scalers: one for the inputs & one for the output
ScalerX = StandardScaler()

#Seperate the train & test dataset into their inputs and output & perform scaling
x_train = ScalerX.fit_transform(train[Columns_Edit])
y_train = train[['Predicted Adj Close']]
x_test = ScalerX.transform(test[Columns_Edit])
y_test = test[['Predicted Adj Close']]

#Intiatize the Polynomial Features & apply it on the Model inputs
Polynomial = PolynomialFeatures(degree=polynomial_degree)
x_train = Polynomial.fit_transform(x_train)
x_test = Polynomial.fit_transform(x_test)

### Run the Primary Linear Regression Model using Final Parameters

In [100]:
#Initialize the Model
Model = LinearRegression(n_jobs=-1)

#Fit the Training Inputs and Output into the Model
Model.fit(x_train, np.array(y_train).ravel())

#Using the Test Inputs, Predict Values for Output
Prediction = Model.predict(x_test)

#Create a Dataframe that stores the Date, the Actual Stock Value and the Predicted Stock Value
Prediction = pd.DataFrame({'Date': test['Date'], 'Actual Value': test['Predicted Adj Close'], 'Prediction': Prediction})

#Calculate the Absolute Difference in Actual Stock Price
Prediction['Predicted Change'] = Prediction['Actual Value'].diff(periods=1)
Prediction['Predicted Change'] = Prediction['Predicted Change'].abs()

#Display the first 5 rows of this Dataframe
Prediction.head()

,Date,Actual Value,Prediction,Predicted Change
877,2018-05-27,183.560894,183.709449,NaN
878,2018-05-28,183.394974,183.606060,0.165920
879,2018-05-29,183.004562,183.392529,0.390411
880,2018-05-30,182.389633,182.978926,0.614929
881,2018-05-31,185.678833,182.343794,3.289200


### Run the Algorithmic Trading Bot

In [101]:
#Set the Current Day & Next Day Date / Predicted Price & Actual Price to their intial values
CurrentDate = Prediction['Date'].values[0]
CurrentPrice = Prediction['Actual Value'].values[0]
NextDayPrediction = Prediction['Prediction'].values[1]
NextDayActual = Prediction['Actual Value'].values[1]

#Save the moeney you started with
InitialMoney = Money

#Loop through all dates in the test set until the 30th of October (-60 days)
for i in range(0, len(Prediction)-61):
    #Run the algorithm
    BuySellStock()
    #Update the Current Day & Next Day Date / Predicted Price & Actual Price values
    CurrentDate = Prediction['Date'].values[i]
    CurrentPrice = Prediction['Actual Value'].values[i]
    NextDayPrediction = Prediction['Prediction'].values[i+1]
    NextDayActual = Prediction['Actual Value'].values[i+1]
    
#Set the date to become date format removing time values    
DateToday = pd.to_datetime(CurrentDate, format='%Y-%m-%d').date()

#On the last day loop through all the stocks that are still owned
for v in range(len(Stocks)-1, -1, -1):
    #Set the date to become date format removing time values   
    DateToday = pd.to_datetime(CurrentDate, format='%Y-%m-%d').date()
    #Sell Stocks, Delete from DataFrame and Append Money
    Money = Money + (CurrentPrice * Stocks['Share Count'].values[v])
    Profit = (CurrentPrice*Stocks['Share Count'].values[v]) - (Stocks['Buy Price'].values[v]*Stocks['Share Count'].values[v])
    print(str(DateToday)+': '+str(int(Stocks['Share Count'].values[v]))+' Shares Sold for: $'+str(round(CurrentPrice, 2))+' Each. Profit Made: $'+str(round(Profit, 2)))
    Stocks.drop(Stocks.index[v], inplace=True)
    
#Print the Total Money at the End
print("Total Money at the End: $"+str(round(Money, 2))+" Profit Made: $"+str(round(Money-InitialMoney, 2)))


2018-05-27: 1 Shares Bought for: $183.56 Each, Total: $183.56
2018-05-27: 1 Shares Bought for: $183.56 Each, Total: $183.56
2018-05-28: 1 Shares Sold for: $183.39 Each. Profit Made: $-0.17
2018-05-28: 1 Shares Sold for: $183.39 Each. Profit Made: $-0.17
2018-05-31: 2 Shares Bought for: $185.68 Each, Total: $371.36
2018-06-01: 1 Shares Bought for: $186.2 Each, Total: $186.2
2018-06-02: 2 Shares Bought for: $186.71 Each, Total: $373.43
2018-06-03: 1 Shares Bought for: $187.23 Each, Total: $187.23
2018-06-04: 1 Shares Bought for: $188.68 Each, Total: $188.68
2018-06-05: 1 Shares Bought for: $189.33 Each, Total: $189.33
2018-06-06: 1 Shares Sold for: $188.82 Each. Profit Made: $-0.51
2018-06-07: 1 Shares Sold for: $187.1 Each. Profit Made: $-1.57
2018-06-07: 1 Shares Sold for: $187.1 Each. Profit Made: $-0.13
2018-06-08: 1 Shares Bought for: $186.95 Each, Total: $186.95
2018-06-09: 1 Shares Bought for: $186.8 Each, Total: $186.8
2018-06-10: 1 Shares Sold for: $186.65 Each. Profit Made: $-0

2018-10-23: 1 Shares Sold for: $210.67 Each. Profit Made: $-5.01
2018-10-23: 1 Shares Sold for: $210.67 Each. Profit Made: $-4.13
2018-10-24: 2 Shares Bought for: $215.28 Each, Total: $430.57
2018-10-25: 2 Shares Sold for: $211.85 Each. Profit Made: $-6.86
2018-10-26: 1 Shares Bought for: $210.53 Each, Total: $210.53
2018-10-27: 1 Shares Sold for: $209.2 Each. Profit Made: $-1.33
2018-10-28: 1 Shares Sold for: $207.88 Each. Profit Made: $-1.06
2018-10-29: 1 Shares Bought for: $208.92 Each, Total: $208.92
2018-10-30: 1 Shares Sold for: $214.36 Each. Profit Made: $5.45
2018-10-30: 1 Shares Sold for: $214.36 Each. Profit Made: $8.44
2018-10-30: 1 Shares Sold for: $214.36 Each. Profit Made: $8.92
2018-10-30: 1 Shares Sold for: $214.36 Each. Profit Made: $9.78
2018-10-30: 1 Shares Sold for: $214.36 Each. Profit Made: $10.22
2018-10-30: 1 Shares Sold for: $214.36 Each. Profit Made: $10.66
2018-10-30: 1 Shares Sold for: $214.36 Each. Profit Made: $12.08
2018-10-30: 5 Shares Sold for: $214.36 